# Assignement: Segmenting and Clustering Neighborhoods in Toronto</font></h1>

## Introduction

"In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto. However, unlike New York, the neighborhood data is not readily available on the internet. What is interesting about the field of data science is that each project can be challenging in its unique way, so you need to learn to be agile and refine the skill to learn new libraries and tools quickly depending on the project. For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset. Once the data is in a structured format, you can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto. Your submission will be a link to your Jupyter Notebook on your Github repository."

## Table of Contents

1. <a href="#item1">Import necessary Libraries</a>
2. <a href="#item2">Create Panda dataframe</a>  
3. <a href="#item3">Web scraping using the BeautifulSoup package or Pandas</a>  
4. <a href="#item4">Get the latitude and the longitude coordinates of each neighborhood</a>  
5. <a href="#item5">Explore and cluster the neighborhoods in Toronto</a>  

<a id='item1'></a>
## 1. Import necessary Libraries
Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [2]:
!pip install geocoder
!pip install lxml

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import geocoder # import geocoder
import folium # map rendering library
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import matplotlib.cm as cm    # Matplotlib and associated plotting modules
import matplotlib.colors as colors
from sklearn.cluster import KMeans  # import k-means from clustering stage

print('Libraries imported.')

Libraries imported.


<a id='item2'></a>
## 2. Objective: Create Panda dataframe
"Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe. [...] For scraping the above table, you can simply use pandas to read the table into a pandas dataframe."

#### (Option: manual and inelegant way) Copy-paste relevant data into "Exel", save as csv-file and read csv-file by pandas.

In [3]:
#!wget -q -O 'postal_codes_toront.csv' https://... csv-file on server
#print('Data downloaded!')
#postal_codes_toronto_df = pd.read_csv('postal_codes_toront.csv') 
#postal_codes_toronto_df.head() 

<a id='item3'></a>
## 3. Web scraping using the BeautifulSoup package or Pandas 
"Another way, which would help to learn for more complicated cases of web scraping is using the BeautifulSoup package. Here is the package's main documentation page: http://beautiful-soup-4.readthedocs.io/en/latest/ . The package is so popular that there is a plethora of tutorials and examples on how to use it. Here is a very good Youtube video on how to use the BeautifulSoup package: https://www.youtube.com/watch?v=ng2o98k983k . Use pandas, or the BeautifulSoup package, or any other way you are comfortable with to transform the data in the table on the Wikipedia page into the above pandas dataframe."

####  Import relevant Libraries

In [4]:
link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
tables = pd.read_html(link)[0]
tables.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


#### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [5]:
df_Toronto=tables[tables['Borough']!="Not assigned"].reset_index(drop=True)
df_Toronto.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


"More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table."

#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [6]:
df_Toronto.loc[(df_Toronto["Neighborhood"]=="NaN"),"Neighborhood"]=df_Toronto["Borough"]
df_Toronto.loc[(df_Toronto["Neighborhood"]=="Business reply mail Processing CentrE"),"Neighborhood"]=df_Toronto["Borough"]
df_Toronto.head(12)

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,Malvern / Rouge
7,M3B,North York,Don Mills
8,M4B,East York,Parkview Hill / Woodbine Gardens
9,M5B,Downtown Toronto,"Garden District, Ryerson"


"Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making."
### In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [7]:
df_Toronto.shape

(103, 3)

#### Submit a link to your Notebook on your Github repository.

<a id='item4'></a>
## 4. Get the latitude and the longitude coordinates of each neighborhood

"Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In an older version of this course, we were leveraging the Google Maps Geocoding API to get the latitude and the longitude coordinates of each neighborhood. However, recently Google started charging for their API: http://geoawesomeness.com/developers-up-in-arms-over-google-maps-api-insane-price-hike/, so we will use the Geocoder Python package instead: https://geocoder.readthedocs.io/index.html.

The problem with this Package is you have to be persistent sometimes in order to get the geographical coordinates of a given postal code. So you can make a call to get the latitude and longitude coordinates of a given postal code and the result would be None, and then make the call again and you would get the coordinates. So, in order to make sure that you get the coordinates for all of our neighborhoods, you can run a while loop for each postal code. Taking postal code M5G as an example, your code would look something like this:"

## Test: Alternative to google geocoder, e.g. Arcgis geocoder

In [8]:
g = geocoder.arcgis('{},Toronto, Canada'.format('M3A'))
latitude=g.latlng[0]
longitude=g.latlng[1]
print(latitude, longitude)

43.75293455500008 -79.33564142299997


In [9]:
def Lat_Long_coordinates_for_Toronto(postal_code): #function to get coordinates
    lat_lng_coords = None     # initialize your variable to None
    while(lat_lng_coords is None):  # loop until you get the coordinates
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords=g.latlng
    latitude=g.latlng[0]
    longitude=g.latlng[1]
    return latitude, longitude    

In [10]:
#df_Toronto['Postal code'][0] #Test
Lat_Long_coordinates_for_Toronto(df_Toronto['Postal code'][0]) #Test of the function

(43.75293455500008, -79.33564142299997)

## Create the dataframe with latitude and longitude coordinate of each neighborhood

In [11]:
df_Toronto_with_coordinates=df_Toronto
df_Toronto_with_coordinates['Latitude'], df_Toronto_with_coordinates['Longitude']=zip(*df_Toronto_with_coordinates['Postal code'].apply(Lat_Long_coordinates_for_Toronto))
df_Toronto_with_coordinates.head(12)

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752935,-79.335641
1,M4A,North York,Victoria Village,43.728102,-79.311890
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.650964,-79.353041
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.723265,-79.451211
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.661790,-79.389390
5,M9A,Etobicoke,Islington Avenue,43.667481,-79.528953
6,M1B,Scarborough,Malvern / Rouge,43.808626,-79.189913
7,M3B,North York,Don Mills,43.748900,-79.357220
8,M4B,East York,Parkview Hill / Woodbine Gardens,43.707193,-79.311529
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657491,-79.377529


<a id='item5'></a>
## 5. Explore and cluster the neighborhoods in Toronto

"You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. It is up to you.

Just make sure:

<ul>
    <li>to add enough Markdown cells to explain what you decided to do and to report any observations you make.</li>
    <li>to generate maps to visualize your neighborhoods and how they cluster together.</li>
</ul>

    
    

Once you are happy with your analysis, submit a link to the new Notebook on your Github repository."

## First Folium Visualisation 

In [12]:
g = geocoder.arcgis('Toronto, Canada')
latitude=g.latlng[0]
longitude=g.latlng[1]
print('The geograpical coordinate of Toronto, Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, Canada are 43.648690000000045, -79.38543999999996.


In [13]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_Toronto_with_coordinates['Latitude'], df_Toronto_with_coordinates['Longitude'], df_Toronto_with_coordinates['Borough'], df_Toronto_with_coordinates['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Work with only boroughs that contain the word Toronto

In [14]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_Toronto_with_coordinates['Borough'].unique()),
        df_Toronto_with_coordinates.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [15]:
df_Toronto_focus=df_Toronto_with_coordinates[df_Toronto_with_coordinates["Borough"].str.contains('Toronto')].reset_index(drop=True)

print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_Toronto_focus['Borough'].unique()),
        len(df_Toronto_focus['Neighborhood'].unique())
    )
)

df_Toronto_focus.head(12)

The dataframe has 4 boroughs and 39 neighborhoods.


,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,Regent Park / Harbourfront,43.650964,-79.353041
1,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.661790,-79.389390
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657491,-79.377529
3,M5C,Downtown Toronto,St. James Town,43.651734,-79.375554
4,M4E,East Toronto,The Beaches,43.678148,-79.295349
5,M5E,Downtown Toronto,Berczy Park,43.645196,-79.373855
6,M5G,Downtown Toronto,Central Bay Street,43.656072,-79.385653
7,M6G,Downtown Toronto,Christie,43.668602,-79.420387
8,M5H,Downtown Toronto,Richmond / Adelaide / King,43.650542,-79.384116
9,M6H,West Toronto,Dufferin / Dovercourt Village,43.664910,-79.438664


In [16]:
# add markers to map
map_toronto2 = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(df_Toronto_focus['Latitude'], df_Toronto_focus['Longitude'], df_Toronto_focus['Borough'], df_Toronto_focus['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto2)  
    
map_toronto2

## Foursquare Analysis

 Credentials hidden/deleted for the sake of date safety.

#### Use the function from the lab on Neighborhoods in Manhattan

In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Use the code to run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [19]:
toronto_venues = getNearbyVenues(names=df_Toronto_focus['Neighborhood'],
                                   latitudes=df_Toronto_focus['Latitude'],
                                   longitudes=df_Toronto_focus['Longitude']
                                 )                                

Regent Park / Harbourfront
Queen's Park / Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond / Adelaide / King
Dufferin / Dovercourt Village
Harbourfront East / Union Station / Toronto Islands
Little Portugal / Trinity
The Danforth West / Riverdale
Toronto Dominion Centre / Design Exchange
Brockton / Parkdale Village / Exhibition Place
India Bazaar / The Beaches West
Commerce Court / Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park / The Junction South
North Toronto West
The Annex / North Midtown / Yorkville
Parkdale / Roncesvalles
Davisville
University of Toronto / Harbord
Runnymede / Swansea
Moore Park / Summerhill East
Kensington Market / Chinatown / Grange Park
Summerhill West / Rathnelly / South Hill / Forest Hill SE / Deer Park
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport
Roseda

#### Let's check the size of the resulting dataframe

In [20]:
print(toronto_venues.shape)
toronto_venues.head()

(781, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Regent Park / Harbourfront,43.650964,-79.353041,Souk Tabule,43.653756,-79.354390,Mediterranean Restaurant
1,Regent Park / Harbourfront,43.650964,-79.353041,Young Centre for the Performing Arts,43.650825,-79.357593,Performing Arts Venue
2,Regent Park / Harbourfront,43.650964,-79.353041,SOMA chocolatemaker,43.650622,-79.358127,Chocolate Shop
3,Regent Park / Harbourfront,43.650964,-79.353041,Cluny Bistro & Boulangerie,43.650565,-79.357843,French Restaurant
4,Regent Park / Harbourfront,43.650964,-79.353041,BATLgrounds,43.647088,-79.351306,Athletics & Sports


#### Let's check how many venues were returned for each neighborhood

In [21]:
toronto_venues.groupby('Neighborhood').count()


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,30,30,30,30,30,30
Brockton / Parkdale Village / Exhibition Place,30,30,30,30,30,30
CN Tower / King and Spadina / Railway Lands / Harbourfront West / Bathurst Quay / South Niagara / Island airport,30,30,30,30,30,30
Central Bay Street,30,30,30,30,30,30
Christie,12,12,12,12,12,12
Church and Wellesley,30,30,30,30,30,30
Commerce Court / Victoria Hotel,30,30,30,30,30,30
Davisville,26,26,26,26,26,26
Davisville North,5,5,5,5,5,5


 
#### Let's find out how many unique categories can be curated from all the returned venues

In [59]:
 print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 179 uniques categories.


## Analyze Each Neighborhood

In [118]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()
print(toronto_onehot.shape)

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
print(toronto_grouped.shape)


(781, 179)
(37, 179)


#### Let's print each neighborhood along with the top 5 most common venues

In [214]:
#num_top_venues = 5

#for hood in toronto_grouped['Neighborhood']:
#    print("----"+hood+"----")
#    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
#    temp.columns = ['venue','freq']
#    temp = temp.iloc[1:]
#    temp['freq'] = temp['freq'].astype(float)
#    temp = temp.round({'freq': 2})
#    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
#    print('\n')

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [259]:
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Seafood Restaurant,Farmers Market,Bakery,Cocktail Bar,Beer Bar,Café,Coffee Shop,Basketball Stadium,Jazz Club,Restaurant
1,Brockton / Parkdale Village / Exhibition Place,Café,Thrift / Vintage Store,Gift Shop,Coffee Shop,Japanese Restaurant,Hawaiian Restaurant,Hotel,Indian Restaurant,Sandwich Place,Italian Restaurant
2,CN Tower / King and Spadina / Railway Lands / ...,Restaurant,Italian Restaurant,Park,Coffee Shop,Beer Bar,Café,Sandwich Place,Seafood Restaurant,Diner,Mexican Restaurant
3,Central Bay Street,Coffee Shop,Plaza,Clothing Store,Japanese Restaurant,Hotel,Shopping Mall,Sandwich Place,Bubble Tea Shop,Restaurant,Italian Restaurant
4,Christie,Grocery Store,Café,Park,Athletics & Sports,Candy Store,Coffee Shop,Baby Store,Playground,Cupcake Shop,Discount Store


   ## Cluster Neighborhoods

In [263]:
# set number of clusters
kclusters = 7


toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([2, 2, 2, 2, 2, 2, 2, 2, 6, 2, 2, 2, 2, 5, 0, 0, 2, 4, 0, 1, 1, 2,
       2, 2, 2, 1, 2, 2, 2, 2, 3, 2, 2, 0, 0, 2, 2], dtype=int32)

In [264]:
#df_Toronto_focus.head()
toronto_merged = df_Toronto_focus

# add clustering labels
neighborhoods_venues_sorted.insert(0,'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2,Berczy Park,Seafood Restaurant,Farmers Market,Bakery,Cocktail Bar,Beer Bar,Café,Coffee Shop,Basketball Stadium,Jazz Club,Restaurant
1,2,Brockton / Parkdale Village / Exhibition Place,Café,Thrift / Vintage Store,Gift Shop,Coffee Shop,Japanese Restaurant,Hawaiian Restaurant,Hotel,Indian Restaurant,Sandwich Place,Italian Restaurant
2,2,CN Tower / King and Spadina / Railway Lands / ...,Restaurant,Italian Restaurant,Park,Coffee Shop,Beer Bar,Café,Sandwich Place,Seafood Restaurant,Diner,Mexican Restaurant
3,2,Central Bay Street,Coffee Shop,Plaza,Clothing Store,Japanese Restaurant,Hotel,Shopping Mall,Sandwich Place,Bubble Tea Shop,Restaurant,Italian Restaurant
4,2,Christie,Grocery Store,Café,Park,Athletics & Sports,Candy Store,Coffee Shop,Baby Store,Playground,Cupcake Shop,Discount Store


In [265]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood').dropna().reset_index(drop=True)
toronto_merged.head() # check the last columns!

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,Regent Park / Harbourfront,43.650964,-79.353041,2.0,Pub,Music Venue,Café,Coffee Shop,Athletics & Sports,Mexican Restaurant,Tech Startup,Seafood Restaurant,Performing Arts Venue,Food Truck
1,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.661790,-79.389390,2.0,Coffee Shop,Sushi Restaurant,Café,Yoga Studio,Middle Eastern Restaurant,Creperie,Park,Diner,Discount Store,Distribution Center
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657491,-79.377529,2.0,Coffee Shop,Café,Ramen Restaurant,Theater,Hotel,Shopping Mall,Comic Shop,Sandwich Place,Burger Joint,Burrito Place
3,M5C,Downtown Toronto,St. James Town,43.651734,-79.375554,2.0,Gastropub,Café,Hotel,Coffee Shop,Cosmetics Shop,Gym,Japanese Restaurant,Italian Restaurant,Farmers Market,Diner
4,M4E,East Toronto,The Beaches,43.678148,-79.295349,0.0,Trail,Health Food Store,Pub,Wine Bar,Department Store,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Distribution Center


In [266]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels'].astype(int)):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters
    
    
    

In [267]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,East Toronto,0.0,Trail,Health Food Store,Pub,Wine Bar,Department Store,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Distribution Center
11,West Toronto,0.0,Bar,Cocktail Bar,Asian Restaurant,Vietnamese Restaurant,Wine Bar,French Restaurant,Park,New American Restaurant,Korean Restaurant,Japanese Restaurant
12,East Toronto,0.0,Park,Grocery Store,Bus Line,Business Service,Discount Store,Dessert Shop,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop
15,East Toronto,0.0,Pizza Place,Fast Food Restaurant,Movie Theater,Restaurant,Italian Restaurant,Steakhouse,Ice Cream Shop,Sushi Restaurant,Pub,Fish & Chips Shop
20,West Toronto,0.0,Tennis Court,Park,Gas Station,Gym / Fitness Center,Sandwich Place,Intersection,Metro Station,Residential Building (Apartment / Condo),Deli / Bodega,Donut Shop


In [268]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Central Toronto,1.0,Park,Gym Pool,Playground,Garden,Deli / Bodega,Electronics Store,Eastern European Restaurant,Donut Shop,Distribution Center,Discount Store
27,Central Toronto,1.0,Park,Gym,Convenience Store,Playground,Department Store,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop,Distribution Center
31,Downtown Toronto,1.0,Park,Grocery Store,Playground,Candy Store,Department Store,Electronics Store,Eastern European Restaurant,Donut Shop,Distribution Center,Discount Store


In [269]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,2.0,Pub,Music Venue,Café,Coffee Shop,Athletics & Sports,Mexican Restaurant,Tech Startup,Seafood Restaurant,Performing Arts Venue,Food Truck
1,Downtown Toronto,2.0,Coffee Shop,Sushi Restaurant,Café,Yoga Studio,Middle Eastern Restaurant,Creperie,Park,Diner,Discount Store,Distribution Center
2,Downtown Toronto,2.0,Coffee Shop,Café,Ramen Restaurant,Theater,Hotel,Shopping Mall,Comic Shop,Sandwich Place,Burger Joint,Burrito Place
3,Downtown Toronto,2.0,Gastropub,Café,Hotel,Coffee Shop,Cosmetics Shop,Gym,Japanese Restaurant,Italian Restaurant,Farmers Market,Diner
5,Downtown Toronto,2.0,Seafood Restaurant,Farmers Market,Bakery,Cocktail Bar,Beer Bar,Café,Coffee Shop,Basketball Stadium,Jazz Club,Restaurant
6,Downtown Toronto,2.0,Coffee Shop,Plaza,Clothing Store,Japanese Restaurant,Hotel,Shopping Mall,Sandwich Place,Bubble Tea Shop,Restaurant,Italian Restaurant
7,Downtown Toronto,2.0,Grocery Store,Café,Park,Athletics & Sports,Candy Store,Coffee Shop,Baby Store,Playground,Cupcake Shop,Discount Store
8,Downtown Toronto,2.0,Coffee Shop,Café,Pizza Place,Seafood Restaurant,Gastropub,Sushi Restaurant,Colombian Restaurant,Lounge,Opera House,Deli / Bodega
9,West Toronto,2.0,Park,Furniture / Home Store,Grocery Store,Bakery,Pharmacy,Brazilian Restaurant,Bank,Pet Store,Athletics & Sports,Gym / Fitness Center
13,Downtown Toronto,2.0,Coffee Shop,Café,Restaurant,Japanese Restaurant,Train Station,Pizza Place,Hotel,Steakhouse,Concert Hall,Art Gallery
